In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import re
from HelperFunctions import *
from PtOccupancyFunctions import *
import io

In [ ]:
def getData(directory):
    end_time = 24*3600 # 24 hours
    for sdir_path, sdir in [(sdir.path, sdir.name) for sdir in os.scandir(directory) if sdir.is_dir()]:
        for root, subdirs, files in os.walk(sdir_path):
            for file in files:
                if file == "output_trips.csv.gz":
                    # TODO analyze servability
                    df_trips = pd.read_csv(
                        os.path.join(root, file), sep=";", compression="gzip"
                    ).loc[:, ["person", "dep_time", "trav_time", "wait_time", "traveled_distance", "modes"]]
                    df_trips["dep_time"] = df_trips["dep_time"].apply(timestmp2sec)
                    df_trips = df_trips[df_trips['dep_time'] < end_time]
                    df_trips["trav_time"] = df_trips["trav_time"].apply(timestmp2sec)
                    df_trips["wait_time"] = df_trips["wait_time"].apply(timestmp2sec)
                    df_trips.set_index("person", inplace=True)
                if file == "0.drt_trips_drt.csv":
                    df_drt = pd.read_csv(os.path.join(root, file), sep=";").loc[
                        :, ["departureTime", "personId", "waitTime", "travelDistance_m", "direcTravelDistance_m"]
                    ]
                    df_drt = df_drt[df_drt['departureTime'] < end_time]
                    df_drt = df_drt.groupby("personId").sum()
                if file == "0.drt_occupancy_time_profiles_drt.txt":
                    df_drt_occ = pd.read_csv(os.path.join(root, file), sep="\t")
                    df_drt_occ['time'] = df_drt_occ['time'].apply(timestmphm2sec)
                    df_drt_occ = df_drt_occ[df_drt_occ['time'] < end_time]
                if file == "0.occupancyAnalysis.txt":
                    df_pt_occ = getPtOccupancies(os.path.join(root, file), 540)
                    df_pt_occ = df_pt_occ[:end_time]
                if file == "trip_success.csv":
                    df_serv = pd.read_csv(os.path.join(root, file), sep=';', header=None)
                    servability = len(df_serv[df_serv[1]])/len(df_serv)
                    

        yield int(sdir), df_trips, df_drt, df_drt_occ, df_pt_occ, servability

In [ ]:
def getAverageOcc(df):
    weights = np.zeros(len(df.columns))
    weights[1:] = np.arange(0,len(df.columns)-1)
    
    pass_sum = df.sum(axis=1)
    mean = (df.dot(weights)/pass_sum).mean()
    s = (df.to_numpy() * (weights - mean)**2).sum(axis=1)/(pass_sum-1)
    return mean, np.sqrt(s).mean()

In [ ]:
path = "/home/helge/Applications/matsim/matsim_results/OwnIntermodalAccEgr/OptDrtCount/WithServability/bim_diag_30min/"

count = 0
vehicle_list = []
av_drt_occs = []
av_pt_occs = []
av_detours = []
total_dists = []
servabilities = []
for vehicles, df_trips, df_drt_trips, df_occ_drt, df_occ_pt, servability in getData(path):
    print('Number of vehicles: ', vehicles)
    df_trips = df_trips.groupby("person").agg(
        {
            "dep_time": lambda x: x.iloc[0],
            "trav_time": np.sum,
            "wait_time": np.sum,
            "traveled_distance": np.sum,
            "modes": combineModesSeriesStr,
        }
    )
    vehicle_list.append(vehicles)
    drt_occ, drt_deviation = getAverageOcc(df_occ_drt.drop(columns = 'time'))
    av_drt_occs.append([drt_occ, drt_deviation])
    av_detours.append(
        (
            df_drt_trips["travelDistance_m"] / df_drt_trips["direcTravelDistance_m"]
        ).mean()
    )
    total_dists.append(df_trips['traveled_distance'].sum())
    av_pt_occs.append(getAverageTimeSeries(df_occ_pt))
    servabilities.append(servability)
        
vehicle_list = np.array(vehicle_list)
av_drt_occs = np.array(av_drt_occs)
av_pt_occs = np.array(av_pt_occs)
av_detours = np.array(av_detours)
total_dists = np.array(total_dists)
servabilities = np.array(servabilities)

In [ ]:
%matplotlib inline
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4,1)

ax1.scatter(vehicle_list, av_drt_occs[:,0]/av_detours)
ax1.set_ylabel(r'$\langle b\rangle / \langle\delta\rangle$')
# ax1.set_xlabel('Number of drt vehicles')
ax2.scatter(vehicle_list, total_dists)
ax2.set_ylabel('total driven distance')
# ax2.set_xlabel('Number of drt vehicles')
ax3.scatter(vehicle_list, av_pt_occs)
ax3.set_ylabel(r'$\langle b_{\mathrm{pt}}\rangle$')
# ax3.set_xlabel('Number of drt vehicles')
ax4.scatter(vehicle_list, servabilities)
ax4.set_ylabel(r'servability')
ax4.set_xlabel('Number of drt vehicles')
fig.tight_layout()

# fig.savefig('OptDrtCount.png', dpi=200)
plt.figure()
plt.scatter(vehicle_list, av_drt_occs[:,0])
plt.scatter(vehicle_list, av_drt_occs[:,1])
plt.show()